<a href="https://colab.research.google.com/github/boleamol/product_sentiment_analysis/blob/main/product_sentiment_analysis_using_Bidirectional_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pwd

/content


In [3]:
#changing the working directory and entering to dataset directory for my google drive
%cd drive/MyDrive/kaggle/product_sentiment_analysis/
!pwd

/content/drive/MyDrive/kaggle/product_sentiment_analysis
/content/drive/MyDrive/kaggle/product_sentiment_analysis


In [4]:
!ls

Test.csv  Train.csv


In [6]:
import tensorflow as tf

In [7]:
tf.__version__

'2.1.0'

In [8]:
import pandas as pd

In [9]:
df=pd.read_csv('Train.csv')

In [10]:
df.head(10)

,Text_ID,Product_Description,Product_Type,Sentiment
0,3057,The Web DesignerÛªs Guide to iOS (and Android...,9,2
1,6254,RT @mention Line for iPad 2 is longer today th...,9,2
2,8212,Crazy that Apple is opening a temporary store ...,9,2
3,4422,The lesson from Google One Pass: In this digit...,9,2
4,5526,RT @mention At the panel: &quot;Your mom has a...,9,2
5,6064,RT @mention I think my effing hubby is in line...,6,1
6,7713,&quot;Android users are not iPhone users. (The...,9,2
7,2975,Wow! RT@mention We interrupt your regularly sc...,9,3
8,818,Google to Launch New Social Network Called Cir...,9,2
9,1318,@mention Welcome! Enjoy #Sxsw and ride anywhe...,9,2


In [11]:
###Drop Nan Values
df=df.dropna()

In [12]:
## Getting Independent Features in X
X=df.drop('Sentiment',axis=1)

In [13]:
## Get the Dependent features
y=df['Sentiment']

In [14]:
X.shape

(6364, 3)

In [15]:
y.shape

(6364,)

In [16]:
y.value_counts()

2    3765
3    2089
1     399
0     111
Name: Sentiment, dtype: int64

In [17]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [18]:
### Vocabulary size
voc_size=5000

In [19]:
messages=X.copy()

In [20]:
messages.reset_index(inplace=True)

In [21]:
import nltk
import re
from nltk.corpus import stopwords

In [22]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [23]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    #print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['Product_Description'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [24]:
corpus

['web design guid io android app today mention link sxsw',
 'rt mention line ipad longer today yesterday sxsw get line today fun',
 'crazi appl open temporari store austin tomorrow handl rabid sxsw eye pad seeker',
 'lesson googl one pass digit environ user want purchas across everi platform one tool sxsw elonsxsw',
 'rt mention panel quot mom ipad design boomer quot sxsw',
 'rt mention think ef hubbi line ipad someon point toward line wife number sxswi sxsw',
 'quot android user iphon user use option menu contextu menu quot mention sxsw',
 'wow rt mention interrupt regularli schedul sxsw geek program big news link googl circl',
 'googl launch new social network call circl possibl today link link sxsw',
 'mention welcom enjoy sxsw ride anywher austin dwnld groundlink app link booth',
 'rt mention appl plan keep austin wire open pop appl store sxsw link',
 'yeay rt mention new ubersoci iphon app store includ uberguid sxsw sponsor masha link',
 'know sxsw season app updat whenev open iph

In [25]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[4512, 1269, 3115, 779, 864, 2804, 2268, 808, 2136, 1738],
 [823, 808, 380, 4891, 3510, 2268, 1846, 1738, 2958, 380, 2268, 852],
 [2021, 2174, 2362, 2745, 4152, 4194, 2821, 2992, 2319, 1738, 3261, 751, 329],
 [63,
  2286,
  2246,
  1086,
  4275,
  1842,
  4124,
  1687,
  344,
  311,
  2704,
  956,
  2246,
  2914,
  1738,
  1899],
 [823, 808, 3343, 2182, 3027, 4891, 1269, 2495, 2182, 1738],
 [823,
  808,
  2716,
  3199,
  3399,
  380,
  4891,
  672,
  786,
  2830,
  380,
  3793,
  3455,
  3687,
  1738],
 [2182, 864, 4124, 2946, 4124, 2062, 3525, 2362, 2889, 2362, 2182, 808, 1738],
 [2929,
  823,
  808,
  1586,
  3066,
  270,
  1738,
  4653,
  1417,
  2386,
  2407,
  2136,
  2286,
  4271],
 [2286, 4441, 4811, 1064, 2908, 4170, 4271, 3680, 2268, 2136, 2136, 1738],
 [808, 4178, 262, 1738, 268, 2146, 4194, 1904, 4046, 2804, 2136, 1205],
 [823, 808, 2174, 3698, 1148, 4194, 3720, 2362, 2170, 2174, 4152, 1738, 2136],
 [4289,
  823,
  808,
  4811,
  2137,
  2946,
  2804,
  4152,
  987,
  2502,

In [26]:
sent_length=30
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  808 2136 1738]
 [   0    0    0 ...  380 2268  852]
 [   0    0    0 ... 3261  751  329]
 ...
 [   0    0    0 ...  320 2136 1738]
 [   0    0    0 ... 2286 3687 1738]
 [   0    0    0 ... 2908 1738 2900]]


In [27]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0, 4512, 1269,
       3115,  779,  864, 2804, 2268,  808, 2136, 1738], dtype=int32)

In [63]:
## Creating model
embedding_vector_features=70
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(200)))
model1.add(Dropout(0.2))
model1.add(Dense(4,activation='softmax'))  # added 4 here because we want 4 classes as output
model1.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 30, 70)            350000    
_________________________________________________________________
bidirectional_6 (Bidirection (None, 400)               433600    
_________________________________________________________________
dropout_6 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 1604      
Total params: 785,204
Trainable params: 785,204
Non-trainable params: 0
_________________________________________________________________
None


In [64]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [65]:
X_final.shape,y_final.shape

((6364, 30), (6364,))

In [66]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [67]:
### Training Model
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=15,batch_size=64)

Train on 4263 samples, validate on 2101 samples
Epoch 1/15
4263/4263 [==============================] - 21s 5ms/sample - loss: 0.9614 - accuracy: 0.5766 - val_loss: 0.9000 - val_accuracy: 0.6121
Epoch 2/15
4263/4263 [==============================] - 18s 4ms/sample - loss: 0.8429 - accuracy: 0.6273 - val_loss: 0.8474 - val_accuracy: 0.6287
Epoch 3/15
4263/4263 [==============================] - 18s 4ms/sample - loss: 0.6720 - accuracy: 0.7302 - val_loss: 0.8650 - val_accuracy: 0.6449
Epoch 4/15
4263/4263 [==============================] - 18s 4ms/sample - loss: 0.5314 - accuracy: 0.7891 - val_loss: 0.9443 - val_accuracy: 0.6487
Epoch 5/15
4263/4263 [==============================] - 17s 4ms/sample - loss: 0.4410 - accuracy: 0.8245 - val_loss: 0.9464 - val_accuracy: 0.6568
Epoch 6/15
4263/4263 [==============================] - 17s 4ms/sample - loss: 0.3845 - accuracy: 0.8506 - val_loss: 1.0610 - val_accuracy: 0.6159
Epoch 7/15
4263/4263 [==============================] - 18s 4ms/sample

In [68]:
y_pred=model1.predict_classes(X_test)

In [69]:
from sklearn.metrics import confusion_matrix

In [70]:
confusion_matrix(y_test,y_pred)

array([[  2,   5,  23,  11],
       [ 13,  32,  51,  26],
       [ 25,  46, 925, 290],
       [  9,  34, 274, 335]])

In [71]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6158971918134222